<a href="https://colab.research.google.com/github/nanac0n/2023F-Ajou-CS/blob/main/%08ML_T_F_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/'path'

/bin/bash: line 1: cd: /content/drive/path: No such file or directory


In [ ]:
import pandas as pd

# Google Drive에 저장된 CSV 파일 경로
file_path = '/content/drive/MyDrive/MBTI_500.csv'

# CSV 파일 로드
df = pd.read_csv(file_path)

# 기존에 T 또는 F를 포함하는 성격 유형에 대해 해당 문자로 대체
df['type'] = df['type'].replace({
    'INTJ': 'T', 'INTP': 'T', 'ENTJ': 'T', 'ENTP': 'T',
    'ISTJ': 'T', 'ISTP': 'T', 'ESTJ': 'T', 'ESTP': 'T',
    'INFJ': 'F', 'INFP': 'F', 'ENFJ': 'F', 'ENFP': 'F',
    'ISFJ': 'F', 'ISFP': 'F', 'ESFJ': 'F', 'ESFP': 'F'
})

# 데이터 프레임 확인
df.head()

,posts,type
0,know intj tool use interaction people excuse a...,T
1,rap music ehh opp yeah know valid well know fa...,T
2,preferably p hd low except wew lad video p min...,T
3,drink like wish could drink red wine give head...,T
4,space program ah bad deal meing freelance max ...,T


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# 텍스트 데이터와 라벨 추출
texts = df['posts'].values
labels = df['type'].values

# 텍스트 토큰화
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# 패딩 적용
maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)

# 라벨 인코딩 (T -> 0, F -> 1)
label_dict = {'T': 0, 'F': 1}
labels = np.array([label_dict[label] for label in labels])

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
pip install --upgrade tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# LSTM 모델 구축
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 253s 117ms/step - accuracy: 0.7062 - loss: 0.5774 - val_accuracy: 0.7913 - val_loss: 0.4658
Epoch 2/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 247s 116ms/step - accuracy: 0.7954 - loss: 0.4536 - val_accuracy: 0.8041 - val_loss: 0.4378
Epoch 3/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 247s 116ms/step - accuracy: 0.8235 - loss: 0.4037 - val_accuracy: 0.8041 - val_loss: 0.4401
Epoch 4/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 249s 117ms/step - accuracy: 0.8399 - loss: 0.3733 - val_accuracy: 0.8024 - val_loss: 0.4503
Epoch 5/5
2122/2122 ━━━━━━━━━━━━━━━━━━━━ 249s 117ms/step - accuracy: 0.8565 - loss: 0.3394 - val_accuracy: 0.7939 - val_loss: 0.4759


In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

# 예측 함수
def predict_mbti(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=maxlen)
    prediction = model.predict(padded_sequence)
    return 'T' if prediction < 0.5 else 'F'

# 예측 예시
sample_text = "I'm so sad"
predicted_type = predict_mbti(sample_text)
print(f'Predicted MBTI type: {predicted_type}')


663/663 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - accuracy: 0.7813 - loss: 0.4917
Test Accuracy: 0.7819
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
Predicted MBTI type: F
